In [2]:
import numpy as np
import keras
from keras.models import Sequential
import matplotlib.pylab as plt
from keras.layers import Dense, Dropout
from keras import utils
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten
from keras.datasets import cifar10
from keras import backend as K
from keras.utils import np_utils

class CuteCNN:
    def __init__(self):
        self.model=Sequential()
        
    def build(self,loss='MSE', optimizer='adam',activation='relu'):
        self.model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(32, 32, 3), activation=activation,padding='same'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(32, 32, 1), activation=activation,padding='same'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(16, 16, 1), activation=activation,padding='same'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(8, 8, 1), activation=activation,padding='same'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Flatten())
        self.model.add(Dense(64, activation=activation))
        self.model.add(Dense(10, activation='softmax'))
        self.model.compile(loss=loss, optimizer=optimizer , metrics=['accuracy'])
        self.model.summary()
        return self.model
    
    def train_model(self,x_train, y_train, _loss='MSE', _optimizer='adam',_activation='relu',
                    _validation_split=0.2, _batch_size=100, _epochs=1, _verbose=2):
        self.build(loss=_loss, optimizer=_optimizer)
        self.model.fit(x_train, y_train, validation_split=_validation_split, 
                       batch_size=_batch_size, epochs=_epochs, verbose=_verbose)
        
        
        
    def evaluate_model(self,x_test,y_test):
        if len(self.model.layers) == 0:
            print("The model hasn't been built.")
            return
        loss, accuracy = self.model.evaluate(x_test,y_test)
        print("The test loss of is %f, the accuracy is %f" % (loss, accuracy))
        return loss, accuracy
    
    def save_model(self,filename):
        self.model.save(filename)
    
    def load_weights(self,filename):
        if len(self.model.layers) == 0:
            self.build()
        self.model.load_weights(filename)
        
    def picture_predict(self,img):
        if len(self.model.layers) == 0:
            print("The model hasn't been built.")
            return
        classes = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]
        scores = self.model.predict(img)
        catagory = np.argmax(scores)
        score = scores[0][catagory]       
        print("Class: "+classes[catagory])
        print("Probability: "+ str(score))
        return score, classes[catagory]
      
    

In [4]:
from keras.utils import np_utils
from keras.datasets import cifar10
import cv2
import matplotlib.pyplot as plt

def load_cifar_dataset():
    print("Load cifar10 dataset...")
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    print('x_train shape:', x_train.shape)
    print('y_test shape:', y_test.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    print("------------------------------")
    y_train = np_utils.to_categorical(y_train)
    y_test = np_utils.to_categorical(y_test)
    
    return x_train/255, y_train, x_test/255, y_test

def classify_picture(img_filename,network):
    img = cv2.imread(img_filename)
    img = cv2.resize(img, (32,32))
    #plot images
    fig = plt.figure(figsize=(5,2))    
    img1 = img.reshape((1,32,32,3))
    score,result = network.picture_predict(img1)
    plt.title("Class: "+result+" Probability: "+ str(score))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    return score,result



In [ ]:
LOSS = 'MSE'
OPTIMIZER = 'adam'
ACTIVATION = 'relu'
VALIDATION_SPLIT=0.2
BATCH_SIZE=100
EPOCHS=10

save_model_path=""
load_model_path=""

img_path=""

if __name__=="__main__":
    
    x_train, y_train, x_test, y_test = load_cifar_dataset()
    network = CuteCNN()
    if load_model_path == "":
        network.train_model(x_train,y_train,_loss=LOSS, _optimizer=OPTIMIZER,_activation=ACTIVATION,
                    _validation_split=VALIDATION_SPLIT, _batch_size=BATCH_SIZE, _epochs=EPOCHS, _verbose=2)
        if save_model_path != "":
            network.save_model(save_model_path)
    else:
        print("Loading Weights from existing file...")
        network.load_weights(load_model_path)
    network.evaluate_model(x_test,y_test)
    
    if img_path !="":
        classify_picture(img_path,network)
    del x_train, y_train, x_test, y_test

Load cifar10 dataset...
x_train shape: (50000, 32, 32, 3)
y_test shape: (10000, 1)
50000 train samples
10000 test samples
------------------------------
Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 32)          9248      
___________________________